Geographical points being key data, but also coming in several different data structures will require an abstraction to deal with it easier. We will implement this idea throuhg a python class GeoPoint:

In [290]:
import pyproj

class GeoType(object):
    def __init__(self, name, values, projparam):
        self.name = name
        self.values = values
        self.projparam = projparam
        
    def getProj(self, geo_point):
        proj = self.projparam["proj"]
        
        for k,v in self.projparam.iteritems():
            for val in self.values:
                self.projparam[k] = geo_point.data[v] if v==val else v
                    
        # add additional parameters here
        self.projparam["datum"] = "WGS84"
        
        proj_param = self.projparam;
        p = pyproj.Proj(**proj_param)            
        
        return p
    
    def transform(self, geo_point, to_geo_type):
        p_from = self.getProj(geo_point)
        p_to = to_geo_type.getProj(geo_point)
        # need to add function that is specific to geo_type. Problem comes from dealing with coord type values
        # x, y = geo_point.data[self.values[0]], geo_point.data[self.values[1]] # TODO: make function for this?
        x,y = self.getargs(geo_point)
        
        out1, out2 = pyproj.transform(p_from, p_to, x, y)
        
        out3, out4 = to_geo_type.postprocess((out1, out2))
        
        return out3, out4
    
    def postprocess(selg, transformed_points):
        return transformed_points
    
    def getargs(self, geo_point):
        return self.getargshelper(geo_point)
    
    def getargshelper(self, geo_point):
        return geo_point.data[self.values[0]], geo_point.data[self.values[1]]
    
    # these where early protoypes, just here for doc purposes
    def getfirstarg(self, geo_point):
        return geo_point.data[self.values[0]]
    
    def getsecondarg(self, geo_point):
        return geo_point.data[self.values[1]]
        
    

In [300]:
import numpy as np

#realization: utm and latlong are "primitive" geoTypes. DEMType has a different structure, 
# but still same interface as the other geoTypes. It also relies on geo_points interface
class DEMType(GeoType):
    # needed for all coordinate conversions. got some serious bootstrapping going on right here
    utm = GeoType("utm", ["easting", "northing", "zonenum"], {"proj": "utm", "zone": "zonenum"})
    
    def __init__(self, origin):
        self.name = "coord"
        self.values = ['x','y']
        # doing conversion early on will save use from redoing it later, we don't expect our origin to change too much
        self.origin = np.array(origin.to(utm)) 
        self.projparamt = []
    
    #TODO: should be able to clean transform function even more
    def getProj(self, geo_points):
        return utm.getProj(geo_points)
    
    # this function is used internally and externally
    def getargs(self, geo_points):
        origin_easting, origin_northing = self.origin
        # next line should ideally be super.getargs, but we overwrite the fx so not sure if possible
        x, y = geo_points.data[geo_points.geo_type.values[0]], geo_points.data[geo_points.geo_type.values[1]]
        points_easting, points_northing = origin_easting + x, origin_northing - y
        return points_easting, points_northing
    
    def postprocess(self, transformed_points):
        origin_easting, origin_northing = self.origin
        points_easting, points_northing = transformed_points
        x, y = (points_easting - origin_easting, origin_northing - points_northing)
        return x, y
    
    
    def transform(self, geo_points, to_geo_type):
        x,y = geo_point.data[self.values[0]], geo_point.data[self.values[1]]
        
        # if we didnt do conversion eaerlier this is where we would have to do it
        origin_easting, origin_northing = self.origin
        points_easting, points_northing = origin_easting + x, origin_northing - y
        
        out3, out4 = to_geo_type.postprocess((points_easting, points_northing))
        
        return out3, out4
        

In [301]:
class GeoPoint(object):
    def __init__(self, geo_type, *args):
        self.geo_type = geo_type 
        self.data = dict()
    
        counter = 0
        for arg in args:
            #print geo_type.values[counter]
            self.data[geo_type.values[counter]] = arg
            #print arg
            counter+=1
            
    def disp(self):
        for k,v in self.data.iteritems():
            print "%s : %s" % (k, v)
    
    def raw(self):
        return self.data.values()
    
    def values(self):
        # need to make immutable
        return self.data.copy()
    
    def to(self, to_geo_type):
        out1, out2 = self.geo_type.transform(self, to_geo_type)
        
        return out1, out2


In [302]:
utm = GeoType("utm", ["easting", "northing", "zonenum"], {"proj": "utm", "zone": "zonenum"})
latlong = GeoType("latlon", ["longitude", "latitude"], {"proj": "latlong"})
origin = GeoPoint(utm, 291931.3692960277, 4815292.614504369, 12)
(lon,lat) = origin.to(latlong)
print(lat,lon)

('utm', 'latlon', 291931.3692960277, 4815292.614504369)
(43.46162074316451, -113.572018535213)


In [303]:
coord = DEMType(origin)

('utm', 'utm', 291931.3692960277, 4815292.614504369)


In [304]:
coord.origin

array([  291931.36929603,  4815292.61450437])

In [305]:
point2 = GeoPoint(latlong, lon, lat)
print(point2.data)

{'latitude': 43.46162074316451, 'longitude': -113.572018535213}


In [306]:
import json
import pandas as pd
pd.options.display.max_rows = 5

with open('MD10_EVA10_Stn18_Stn23_X.json') as data_file:    
    data = json.load(data_file)
ways_and_segments = data['sequence']
s = pd.DataFrame(ways_and_segments)
waypoints = s[s['type']=='Station']['geometry']
w = waypoints.values.tolist()
latlongFull = pd.DataFrame(w)
latlongInter = latlongFull['coordinates'].values.tolist()
waypointslatlong = pd.DataFrame(latlongInter, columns=['longitude','latitude'])
waypointslatlong

,longitude,latitude
0,-113.572019,43.461621
1,-113.571818,43.462455
...,...,...
9,-113.573593,43.463636
10,-113.574050,43.463874


In [307]:
waypoints = GeoPoint(latlong, waypointslatlong['longitude'].values, waypointslatlong['latitude'].values)

In [308]:
waypoints.to(coord)

check
('latlon', 'coord', array([-113.57201854, -113.57181845, -113.57158672, -113.57174616,
       -113.57174874, -113.57189094, -113.57219385, -113.57266771,
       -113.57305409, -113.5735927 , -113.5740501 ]), array([ 43.46162074,  43.46245467,  43.46283157,  43.46295182,
        43.46319999,  43.46317908,  43.46329632,  43.46331013,
        43.46330418,  43.4636355 ,  43.4638741 ]))


(array([   0.        ,   19.04848272,   39.08867053,   26.60239684,
          27.24575373,   15.6704511 ,   -8.43224242,  -46.71940985,
         -77.99637105, -120.43133923, -156.61426585]),
 array([ -9.31322575e-10,  -9.21189277e+01,  -1.33399649e+02,
         -1.47153399e+02,  -1.74722459e+02,  -1.72755794e+02,
         -1.86533707e+02,  -1.89251676e+02,  -1.89557736e+02,
         -2.27702148e+02,  -2.55346097e+02]))